# TODO

Write description

<a id='dependencies'></a>

## 1. Dependencies

Some libraries might require pip install.

In [ ]:
# standard libraries
import numpy as np
import pandas as pd
import time
import os

import seaborn as sns
sns.set_style("whitegrid")

# machine learning libraries
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb


In [ ]:
# display full output in Notebook, instead of only the last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt

# Define the locations to be used.
locations = ['burgdorf', 'rapperswil']

# Set this to False if another grid-search should be preformed.
use_default = True

# Set this to False if no tex environment is installed.
use_tex = True

# Set this to True if plots should be saved to disk.
save_figs = True


In [ ]:
# Parameterset for iterative results (ONLY RUN ONCE!)

prediction_params = {
    "12h": {
        "horizon": 12,
        "time": "12 hours",
        "label": "+12",
        "features": ['hour','day_of_week','quarter','month','day_of_year','day_of_month','week_of_year','holiday',
                    'occupancy_rate',
                    'temperature',
                    'temperature+12',
                    'weather',
                    'weather+12',
                    ]
    },
    "24h": {
        "horizon": 24,
        "time": "24 hours",
        "label": "+24",
        "features": ['hour','day_of_week','quarter','month','day_of_year','day_of_month','week_of_year','holiday',
                    'temperature+24',
                    'weather+24',
                    ]
    },
    "7d": {
        "horizon": 168,
        "time": "7 days",
        "label": "+168",
        "features": ['hour','day_of_week','quarter','month','day_of_year','day_of_month','week_of_year','holiday',
                    'temperature+168',
                    'weather+168',
                    ]
    },
    "14d": {
        "horizon": 336,
        "time": "14 days",
        "label": "+336",
        "features": ['hour','day_of_week','quarter','month','day_of_year','day_of_month','week_of_year','holiday',
                    'temperature+336',
                    'weather+336',
                    ]
    },
    "28d": {
        "horizon": 672,
        "time": "28 days",
        "label": "+672",
        "features": ['hour','day_of_week','quarter','month','day_of_year','day_of_month','week_of_year','holiday',
                    'temperature+672',
                    'weather+672',
                    ]
    },
    "90d": {
        "horizon": 2160,
        "time": "90 days",
        "label": "+2160",
        "features": ['hour','day_of_week','quarter','month','day_of_year','day_of_month','week_of_year','holiday',
                    'temperature+2160',
                    'weather+2160',
                    ]
    }
}

In [ ]:
# Select horizon from [12h, 24h, 7d, 14d, 28d, 90d]
horizon = "12h"

prediction_param = prediction_params[horizon]

Select a prediction horizon in the following cell. Repeat from there with all keys.

<a id='data_import'></a>

## 2. Data Import

In [ ]:
data_path = '../00_data'

df_rapperswil = pd.read_csv(os.path.join(data_path, "features_rapperswil.csv"), sep=",")
df_burgdorf = pd.read_csv(os.path.join(data_path, "features_burgdorf.csv"), sep=",")

print('Dataset shape of Rapperswil data: {}'.format(df_rapperswil.shape))
print('Dataset shape of Burgdorf data: {}'.format(df_burgdorf.shape))

dfs = {}

for loc in locations:
    dfs[loc] = pd.read_csv(os.path.join(data_path, "features_{}.csv".format(loc)), sep=",")
    dfs[loc]['date'] = pd.to_datetime(dfs[loc]['date'])

    #  declare categorical columns
    for col in ['hour', 'day_of_week', 'quarter', 'month', 'day_of_year', 'day_of_month',
                'week_of_year', 'weather', 'weather_t-1', 'weather_t-2', 'weather_t-3', 'weather_t-7', 
                'holiday']:
        
        dfs[loc][col] = dfs[loc][col].astype(object)

    # set datetime column as index
    dfs[loc].set_index('date', inplace = True)


For the long-term predictions, shift the labels by the specified amount of time and adapt feature extraction

In [ ]:
for loc in locations:
    dfs[loc]['occupancy{}'.format(prediction_param['label'])] = dfs[loc]['occupancy_rate']
    dfs[loc]['temperature{}'.format(prediction_param['label'])] = dfs[loc]['temperature']
    dfs[loc]['weather{}'.format(prediction_param['label'])] = dfs[loc]['weather']
    dfs[loc]['occupancy{}'.format(prediction_param['label'])] = dfs[loc]['occupancy{}'.format(prediction_param['label'])].shift(-prediction_param['horizon'])
    dfs[loc]['temperature{}'.format(prediction_param['label'])] = dfs[loc]['temperature{}'.format(prediction_param['label'])].shift(-prediction_param['horizon'])
    dfs[loc]['weather{}'.format(prediction_param['label'])] = dfs[loc]['weather{}'.format(prediction_param['label'])].shift(-prediction_param['horizon'])
    dfs[loc] = dfs[loc].iloc[:-prediction_param['horizon']]


In [ ]:
## Extracts a feature column from the dataframe.\n",
# @param df         the dataframe.
# @param features   the features to be included.
# @param label      the label of the column to be extracted.
#
# @returns the original dataframe and (if found) the column indexed by @p label or (if not found) None.
def extract_features(df:pd.DataFrame, features:list, label:str=None):
    X = df[features]
    if label:
        y = df[label]
        return X, y
    return X, None

In [ ]:
for loc in locations:
    print('Dataset shape of {} data: {}'.format(loc.capitalize(), dfs[loc].shape))


<a id='data_preprocessing'></a>

## 3. Data Preprocessing

Split datasets into train and test data using 70/30% ratio by considering the order of the time series.

In [ ]:
# identify split date manually and use beginning of the month of the specific date
# e.g. 2021-08-16 04:00:00 --> 2021-08-01 01:00:00, only for pragmatic reasons for better paper story
splits = {}

for loc in locations:
    splits[loc] = 0.7 * len(dfs[loc])


In [ ]:
## Splits a datetime-indexed dataframe according to a specified split date.
# @param df         a datetime-indexed dataframe.
# @param split_date the split date.
#
# @returns A copy of the lower part of the dataframe (including split date) and a copy of the upper part.

def split(df:pd.DataFrame, split_date:pd.DatetimeIndex):
    # split df into train and test set
    df_train = df.loc[df.index <= split_date].copy()
    df_test = df.loc[df.index > split_date].copy()
    
    return df_train, df_test


In [ ]:
dfs_train, dfs_test = {}, {}

for loc in locations:
    dfs_train[loc], dfs_test[loc] = split(dfs[loc], dfs[loc].index[int(splits[loc])])


In [ ]:
for loc in locations:
    print('Cutoff date {}:\t{}\t({}/{} entries)'.format(loc.capitalize(), dfs[loc].index[int(splits[loc])], len(dfs_train[loc]), len(dfs_test[loc])))


Split datasets

In [ ]:
# extract the occupancy rate as a label
X_train, X_test = {}, {}
y_train, y_test = {}, {}

for loc in locations:
    X_train[loc], y_train[loc] = extract_features(dfs_train[loc], prediction_param['features'], label='occupancy{}'.format(prediction_param['label']))
    X_test[loc], y_test[loc] = extract_features(dfs_test[loc], prediction_param['features'], label='occupancy{}'.format(prediction_param['label']))


### Feature standardization and scaling

Features vary in magnitude and units, which is why feature scaling is applied, using `StandardScaler()` for numeric features and `OneHotEncoder()` for categorical features. For example, the input value `day_of_week` should not be used as a continuous value from 1 to 7, since this would associate a higher weight to the later weekdays (5, 6 or 7) than to the earlier ones (1,2 and 3).

In [ ]:
## Standardize features using standard scaling and one-hot encoding.
# @p df_train   the training dataframe.
# @p df_test    the test dataframe.
#
# @returns  the standardized training and test datasets, as well as the generated pipeline object.
def standardize_features(df_train:pd.DataFrame, df_test:pd.DataFrame):
    # split numerical and categorical columns
    data_num = df_train.select_dtypes(include=[np.number])
    data_cat = df_train.select_dtypes(include=[object])

    # check whether no columns got lost during type allocation
    len(df_train.columns) == len(data_num.columns) + len(data_cat.columns)

    # create data pipeline
    num_pipeline = Pipeline([('std_scaler', StandardScaler())])

    num_attribs = list(data_num)
    cat_attribs = list(data_cat)

    full_pipeline = ColumnTransformer([
            ('num', num_pipeline, num_attribs),
            # don’t take precautions to handle unseen values for OneHotEncoder
            ('cat', OneHotEncoder(handle_unknown = 'ignore'), cat_attribs)])

    # fit and transform training data set with preprocessing pipeline and
    # only transform test feature set with the pipeline fit on training feature set to not
    # artificially improve test performance

    return full_pipeline.fit_transform(df_train), full_pipeline.transform(df_test), full_pipeline


In [ ]:
pipelines = {}

for loc in locations:
        X_train[loc], X_test[loc], pipelines[loc] = standardize_features(X_train[loc], X_test[loc])


<a id='model_training'></a>

## 4. Model Training

For hyperparameter optimization, we apply `random grid search` with cross validation to narrow down the range of reasonable values for the given parameters for the models. Then, `full grid search` with cross validation is applied with the value range obtained from the random grid search.

1) Define random grid and run grid search

In [ ]:
t_start, t_end, params = {}, {}, {}

if not use_default:
    # number of trees in random forest
    n_estimators = [x for x in range(200, 2000, 25)]

    # learning rate
    arr = np.arange(0.01, 1.0, 0.05)
    eta = arr.tolist()

    # defines the minimum sum of weights of all observations required in a child
    min_child_weight = [i for i in range(1, 100, 8)]

    # maximum depth of a tree
    max_depth = [i for i in range(1, 250, 5)]

    # regularization
    gamma = [0, 0.5, 1, 3, 5]

    # number of columns used by each tree
    colsample_bytree = [0.5, 0.6, 0.7, 0.8, 0.9, 1]

    # maximum number of leaf nodes in tree
    max_leaf_nodes = [i for i in range(10, 300, 5)]
    max_leaf_nodes.append(None)

    # create random grid
    random_grid = {'n_estimators' : n_estimators,
                   'eta' : eta,
                   'max_depth': max_depth,
                # 'max_features': max_features,
                   'min_child_weight': min_child_weight,
                   'gamma' : gamma,
                   'colsample_bytree' : colsample_bytree,
                   'max_leaf_nodes': max_leaf_nodes}

    for loc in locations:
        # create xgb model
        regressor_rnd = xgb.XGBRegressor(random_state=42)

        # ensure prediction is made on subsequent data
        cv = TimeSeriesSplit(n_splits=3)

        # random search of parameters, using 3 fold cross validation, 
        # search across 75 different combinations, and use all available cores
        xbg_random = RandomizedSearchCV(estimator = regressor_rnd, param_distributions = random_grid,
                                        n_iter = 75, cv = cv,
                                        verbose=2, random_state=42, n_jobs = -1)
        # fit the xgboost model
        t_start[loc] = time.time()
        xbg_random.fit(X_train[loc], y_train[loc])
        t_end[loc] = time.time()

        params[loc] = xbg_random.best_params_


In [ ]:
if not use_default:
    for loc in locations:
        print("Grid-search {} took {} seconds.".format(loc.capitalize(), t_end[loc] - t_start[loc]))
        print("Best parameters are:")
        params[loc]
        print("\n\n")

2. Use same procedure to run the `full_grid_search` with the obtained values from `random_grid_search`

In [ ]:
# provide more granular range based on values from random grid search
if not use_default:
    for loc in locations:

        # number of trees in random forest
        n_estimators = [x for x in range(max(params[loc]['n_estimators'] - 25, 1), params[loc]['n_estimators'] + 25, 1)]
        
        # learning rate
        eta = np.array([params[loc]['eta']]).tolist()

        # defines the minimum sum of weights of all observations required in a child
        min_child_weight = [i for i in range(max(params[loc]['min_child_weight'] - 8, 1), params[loc]['min_child_weight'] + 8, 1)]

        # maximum depth of a tree
        max_depth = [i for i in range(max(params[loc]['max_depth'] - 5, 1), params[loc]['max_depth'] + 5, 1)]
        
        # regularization
        gamma = [0, 1, 5]
        
        # number of columns used by each tree
        colsample_bytree = [0.6, 0.7, 0.8, 0.9, 1]
        
        # maximum number of leaf nodes in tree
        max_leaf_nodes = [i for i in range(max(params[loc]['max_depth'] - 5, 1), params[loc]['max_depth'] + 5, 1)]
        max_leaf_nodes.append(None)
        
        # create random grid
        random_grid = {'n_estimators' : n_estimators,
            'eta' : eta,
            'max_depth': max_depth,
            # 'max_features': max_features,
            'min_child_weight': min_child_weight,
            'gamma' : gamma,
            'colsample_bytree' : colsample_bytree,
            'max_leaf_nodes': max_leaf_nodes}

        # create xgb model
        regressor_rnd = xgb.XGBRegressor(random_state=42)

        # ensure prediction is made on subsequent data
        cv = TimeSeriesSplit(n_splits=3)

        # random search of parameters, using 3 fold cross validation, 
        # search across 75 different combinations, and use all available cores
        xbg_random = RandomizedSearchCV(estimator = regressor_rnd, param_distributions = random_grid,
                                        n_iter = 75, cv = cv,
                                        verbose=2, random_state=42, n_jobs = -1)

        # xbg_random = GridSearchCV(estimator = regressor_rnd, param_distributions = random_grid,
        #                                 n_iter = 75, cv = cv,
        #                                 verbose=2, random_state=42, n_jobs = -1)

        # fit the xgboost model
        t_start[loc] = time.time()
        xbg_random.fit(X_train[loc], y_train[loc])
        t_end[loc] = time.time()

        params[loc] = xbg_random.best_params_

else:
    params['burgdorf'] = {
        'n_estimators': 225,
        'min_child_weight': 17,
        'max_leaf_nodes': 285,
        'max_depth': 241,
        'gamma': 0.5,
        'eta': 0.16000000000000003,
        'colsample_bytree': 0.7
    }
        
    params['rapperswil'] = {
        'n_estimators': 550,
        'min_child_weight': 73,
        'max_leaf_nodes': 25,
        'max_depth': 36,
        'gamma': 1,
        'eta': 0.21000000000000002,
        'colsample_bytree': 0.5
    }


In [ ]:
if not use_default:
    for loc in locations:
        print("Grid-search {} took {} seconds.".format(loc.capitalize(), t_end[loc] - t_start[loc]))
        print("Best parameters are:")
        params[loc]
        print("\n\n")


In [ ]:
regressors = {}

for loc in locations:
    regressors[loc] = xgb.XGBRegressor(**params[loc], random_state=42)

    # fit the random search model
    t_start[loc] = time.time()
    _ = regressors[loc].fit(X_train[loc], y_train[loc])
    t_end[loc] = time.time()


In [ ]:
for loc in locations:
    print("Fitting for {} took {} seconds.".format(loc.capitalize(), t_end[loc] - t_start[loc]))


3. Run predictions

In [ ]:
maes, rmses = {}, {}

for loc in locations:
    t_start[loc] = time.time()
    dfs_test[loc]['pred_xgb_all_features'] = regressors[loc].predict(X_test[loc])
    t_end[loc] = time.time()
    maes[loc] = mean_absolute_error(y_true=dfs_test[loc]['occupancy{}'.format(prediction_param['label'])], y_pred=dfs_test[loc]['pred_xgb_all_features'])
    rmses[loc] = mean_squared_error(y_true=dfs_test[loc]['occupancy{}'.format(prediction_param['label'])], y_pred=dfs_test[loc]['pred_xgb_all_features'], squared=False)


In [ ]:
for loc in locations:
    print("Prediction for {} ({} entries) took {} seconds.".format(loc.capitalize(), len(dfs_test[loc]), round(t_end[loc] - t_start[loc], 2)))
    print("\tMAE: ", round(maes[loc], 2))
    print("\tRMSE: ", round(rmses[loc], 2))


4. Plot results

In [ ]:
plt_params = {
    'text.usetex' : use_tex,
    'font.size' : 20,
    'xtick.labelsize' : 18,
    'ytick.labelsize' : 18,
    'lines.linewidth': 1,
    'grid.linewidth':   2,
}
plt.rcParams.update(plt_params)

for loc in ['burgdorf', 'rapperswil']:
    fig = plt.figure(figsize=(20,8))

    # plot the true target values for the test set versus the estimated values with the best model
    _ = plt.scatter(dfs_test[loc]['occupancy{}'.format(prediction_param['label'])], 
        dfs_test[loc]['pred_xgb_all_features'],
        alpha=0.1);
    _ = plt.plot([0, 100], [0, 100], "k--", lw=3);
    _ = plt.xlabel(r'Measured parking occupancy $\left[\%\right]$');
    _ = plt.ylabel(r'Predicted parking occupancy $\left[\%\right]$');
    _ = plt.title('Prediction for {} days: True values vs predicted values ({})'.format(prediction_param['time'], loc.capitalize()));
    _ = plt.grid(None)

    # provide MAE and RMSE details to the plot
    textstr = '\n'.join((
        r'MAE: %.2f' % (maes[loc]),
        r'RMSE: %.2f' % (rmses[loc])
    ))

    # style infobox
    # plt.gca().hist(x, 20)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    _ = plt.text(0.03, 0.95, textstr, verticalalignment='top', bbox=props, transform=plt.gca().transAxes)

    if save_figs:
        # export - pay attention to an appropriate name
        filename = 'pred_' + loc + prediction_param['label'] + '.png'
        plt.savefig('../02_results/' + filename)


In [ ]:
prediction_params[horizon]['results'] = {}

for loc in locations:
    df_error = round(dfs_test[loc]['occupancy{}'.format(prediction_param['label'])] - dfs_test[loc]['pred_xgb_all_features'], 2)

    # set size of figure
    fig = plt.figure(figsize=(20,12));
    ax = sns.histplot(df_error)

    _ = plt.xlabel(r'Prediction Error $\left[\%\right]$');
    _ = plt.ylabel(r'Number of occurences');
    _ = plt.title(r'Prediction for ' + prediction_param['time'] + r': $\hat{y} - y$ (' + loc.capitalize() + ')');
    _ = plt.grid(None)

    textstr = '\n'.join((
        r'$\mu:$ ' + str(round(np.mean(df_error), 2)),
        r'$\sigma:$ ' + str(round(np.std(df_error), 2))
    ))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    _ = plt.text(0.03, 0.95, textstr, verticalalignment='top', bbox=props, transform=plt.gca().transAxes)

    prediction_params[horizon]['results'][loc] = df_error

    # save fig
    if save_figs:
        filename = 'hist_' + loc + prediction_param['label'] + '.png'
        plt.savefig('../02_results/' + filename)

In [ ]:
df_errors = {}

for loc in locations:

    df_errors[loc] = pd.DataFrame(columns=prediction_params.keys())
    for hor in prediction_params.keys():
        df_errors[loc][hor] = prediction_params[hor]['results'][loc]

    # set size of figure
    fig = plt.figure(figsize=(20,12));

    ax = sns.boxplot(data=df_errors[loc], linewidth=2, color="#B4D0E4", boxprops=dict(alpha=.7));

    x_labels = []
    for hor in prediction_params.keys():
        x_labels.append(prediction_params[hor]['time'])

    _ = ax.set_xticklabels(x_labels)

    _ = plt.title(r'Boxplot of Prediction Error $\hat{y} - y$ ' + '({})'.format(loc.capitalize()))    
    _ = plt.xlabel(r'Prediction Horizon');
    _ = plt.ylabel(r'Prediction Error');

    # save fig
    if save_figs:
        filename = 'box_' + loc + prediction_param['label'] + '.png'
        plt.savefig('../02_results/' + filename)
